## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 6
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_6


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 35.4 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Buffalo Bills,6,2022,29.3,60.0,8.1,323.0,106.9,12.3,13.5,...,10.2,76.2,21.0,12.0,12.0,0.114035,2.833333,0.666667,1.25,4.25
1,Detroit Lions,5,2022,28.0,44.4,7.3,260.4,93.0,12.2,34.0,...,12.0,167.6,8.6,13.0,12.4,0.094488,2.200000,2.000000,0.80,2.75


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[6, 5]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Buffalo Bills,2022,week_6,0.916132,0.376864,0.215874,0.328889,0.413679,0.450287,0.476485
4,Kansas City Chiefs,2022,week_6,0.205226,0.221748,0.173160,0.279385,0.402198,0.256343,0.377577
17,San Francisco 49ers,2022,week_6,0.539541,0.165052,0.108280,0.216718,0.352730,0.276464,0.376081


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Buffalo Bills,2022,week_6,0.916132,0.376864,0.215874,0.328889,0.413679,0.450287,0.476485
4,Kansas City Chiefs,2022,week_6,0.205226,0.221748,0.173160,0.279385,0.402198,0.256343,0.377577
17,San Francisco 49ers,2022,week_6,0.539541,0.165052,0.108280,0.216718,0.352730,0.276464,0.376081
20,Tampa Bay Buccaneers,2022,week_6,0.282847,0.185240,0.128230,0.254023,0.354863,0.241041,0.345861
2,Philadelphia Eagles,2022,week_6,0.164349,0.265670,0.153657,0.267671,0.361131,0.242496,0.336533
26,Dallas Cowboys,2022,week_6,0.124148,0.163814,0.100028,0.208400,0.356825,0.190643,0.327740
21,Denver Broncos,2022,week_6,0.105180,0.163814,0.100187,0.199308,0.346399,0.182978,0.316246
19,Cincinnati Bengals,2022,week_6,0.018018,0.130448,0.107686,0.275707,0.356215,0.177615,0.313941
8,Baltimore Ravens,2022,week_6,0.025129,0.045358,0.044546,0.180941,0.352934,0.129782,0.311958
12,New England Patriots,2022,week_6,0.082282,0.105876,0.071788,0.213131,0.340549,0.162725,0.308266
